In [ ]:
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
wefarm = pd.read_parquet("/content/drive/MyDrive/DataKit_WeFarm/wefarm.parquet")

In [ ]:
wefarm.shape

(20304843, 24)

In [ ]:
wefarm["response_sent_dt"] = pd.to_datetime(wefarm["response_sent"], format="ISO8601")
wefarm["question_set_dt"] = pd.to_datetime(wefarm["question_sent"], format="ISO8601")

In [ ]:
def parse_datetime_col(df, colstr):
  df[f'{colstr}_year'] = df[colstr].dt.year
  df[f'{colstr}_month'] = df[colstr].dt.month
  df[f'{colstr}_day'] = df[colstr].dt.day
  df[f'{colstr}_hour'] = df[colstr].dt.hour
  df[f'{colstr}_minute'] = df[colstr].dt.minute
  df[f'{colstr}_second'] = df[colstr].dt.second
  df[f'{colstr}_day_name'] = df[colstr].dt.day_name()
  df[f'{colstr}_day_of_week'] = df[colstr].dt.day_of_week
  df[f'{colstr}_month_name'] = df[colstr].dt.month_name()
  df[f'{colstr}_quarter'] = df[colstr].dt.quarter
  df[f'{colstr}_week'] = df[colstr].dt.isocalendar().week
  df[f'{colstr}_date'] = df[colstr].dt.date
  df[f'{colstr}_time'] = df[colstr].dt.time

  return df

In [ ]:
wefarm = parse_datetime_col(wefarm, "response_sent_dt")
wefarm = parse_datetime_col(wefarm, "question_set_dt")

In [ ]:
wefarm["question_response_min_diff"] = (wefarm["response_sent_dt"] - wefarm["question_set_dt"]).dt.total_seconds() / 60

In [ ]:
wefarm.head(5)

,question_id,question_user_id,question_language,question_content,question_topic,question_sent,response_id,response_user_id,response_language,response_content,response_topic,response_sent,question_user_type,question_user_status,question_user_country_code,question_user_gender,question_user_dob,question_user_created_at,response_user_type,response_user_status,response_user_country_code,response_user_gender,response_user_dob,response_user_created_at,response_sent_dt,question_set_dt,response_sent_dt_year,response_sent_dt_month,response_sent_dt_day,response_sent_dt_hour,response_sent_dt_minute,response_sent_dt_second,response_sent_dt_day_name,response_sent_dt_day_of_week,response_sent_dt_month_name,response_sent_dt_quarter,response_sent_dt_week,response_sent_dt_date,response_sent_dt_time,question_set_dt_year,question_set_dt_month,question_set_dt_day,question_set_dt_hour,question_set_dt_minute,question_set_dt_second,question_set_dt_day_name,question_set_dt_day_of_week,question_set_dt_month_name,question_set_dt_quarter,question_set_dt_week,question_set_dt_date,question_set_dt_time,question_response_min_diff
0,3849056,519124,nyn,E ABA WEFARM OFFICES ZABO NIZISHANGWA NKAHI?,None,2017-11-22 12:25:03+00,20691011,200868,nyn,E!23 Omubazi Ni Dudu Cipa',None,2019-01-24 17:54:06.216221+00,farmer,live,ug,None,None,2017-11-18 13:09:11+00,farmer,live,ug,None,None,2017-05-09 09:19:33+00,2019-01-24 17:54:06.216221+00:00,2017-11-22 12:25:03+00:00,2019,1,24,17,54,6,Thursday,3,January,1,4,2019-01-24,17:54:06.216221,2017,11,22,12,25,3,Wednesday,2,November,4,47,2017-11-22,12:25:03,616649.053604
1,3849061,521327,eng,Q this goes to wefarm. is it possible to get f...,None,2017-11-22 12:25:05+00,4334249,526113,eng,Q1 which stage is marleks last vaccinated,None,2018-01-04 08:57:28+00,farmer,live,ug,None,None,2017-11-20 11:55:48+00,farmer,zombie,ug,None,None,2017-11-22 10:13:03+00,2018-01-04 08:57:28+00:00,2017-11-22 12:25:05+00:00,2018,1,4,8,57,28,Thursday,3,January,1,1,2018-01-04,08:57:28,2017,11,22,12,25,5,Wednesday,2,November,4,47,2017-11-22,12:25:05,61712.383333
2,3849077,307821,nyn,E ENTE YANJE EZAIRE ENYENA YASHOBERA. \nOBWIRE...,cattle,2017-11-22 12:25:08+00,3849291,296187,nyn,Muhanguzi.Benon kuruga masha isingiro ente yaw...,tomato,2017-11-22 12:35:26+00,farmer,zombie,ug,None,None,2017-08-22 14:51:07+00,farmer,zombie,ug,None,None,2017-08-12 09:30:33+00,2017-11-22 12:35:26+00:00,2017-11-22 12:25:08+00:00,2017,11,22,12,35,26,Wednesday,2,November,4,47,2017-11-22,12:35:26,2017,11,22,12,25,8,Wednesday,2,November,4,47,2017-11-22,12:25:08,10.300000
3,3849077,307821,nyn,E ENTE YANJE EZAIRE ENYENA YASHOBERA. \nOBWIRE...,cattle,2017-11-22 12:25:08+00,3849291,296187,nyn,Muhanguzi.Benon kuruga masha isingiro ente yaw...,cattle,2017-11-22 12:35:26+00,farmer,zombie,ug,None,None,2017-08-22 14:51:07+00,farmer,zombie,ug,None,None,2017-08-12 09:30:33+00,2017-11-22 12:35:26+00:00,2017-11-22 12:25:08+00:00,2017,11,22,12,35,26,Wednesday,2,November,4,47,2017-11-22,12:35:26,2017,11,22,12,25,8,Wednesday,2,November,4,47,2017-11-22,12:25:08,10.300000
4,3849077,307821,nyn,E ENTE YANJE EZAIRE ENYENA YASHOBERA. \nOBWIRE...,cat,2017-11-22 12:25:08+00,3849291,296187,nyn,Muhanguzi.Benon kuruga masha isingiro ente yaw...,tomato,2017-11-22 12:35:26+00,farmer,zombie,ug,None,None,2017-08-22 14:51:07+00,farmer,zombie,ug,None,None,2017-08-12 09:30:33+00,2017-11-22 12:35:26+00:00,2017-11-22 12:25:08+00:00,2017,11,22,12,35,26,Wednesday,2,November,4,47,2017-11-22,12:35:26,2017,11,22,12,25,8,Wednesday,2,November,4,47,2017-11-22,12:25:08,10.300000


In [ ]:
niche = pd.read_parquet("/content/drive/MyDrive/DataKit_WeFarm/en_questions_cat_niche.parquet")
broad = pd.read_parquet("/content/drive/MyDrive/DataKit_WeFarm/en_questions_cat_broad.parquet")

In [ ]:
all = pd.merge(wefarm, niche, on="question_id").merge(
  broad, on="question_id"
)

In [ ]:
wefarm.shape

(20304843, 53)

In [ ]:
import json

with open("/content/drive/MyDrive/DataKit_WeFarm/segment_user_ids.json", "r") as r:
    segment_user_id_dct = json.load(r)

In [ ]:
# segment_user_id_dct.keys()

In [ ]:
# segment_user_id_dct["user_activity_post_count"].keys()

In [ ]:
from collections import defaultdict

user_segments = defaultdict(dict)
for agg, dct in segment_user_id_dct.items():
  for segment, user_ids in dct.items():
    for user_id in user_ids:
      user_segments[user_id][agg] = segment

row_lst = []
for user_id, segments in user_segments.items():
  row = {"user_id": user_id}
  row.update(segments)
  row_lst.append(row)

user_segments_df = pd.DataFrame(row_lst)

In [ ]:
all1 = all.merge(user_segments_df, left_on="question_user_id", right_on="user_id", how="inner")

In [ ]:
all2 = all1[all1["question_user_country_code"].isin(["ug", "ke"])]

In [ ]:
all2["question_response_min_diff_buck"] = pd.qcut(all2["question_response_min_diff"], q=10).cat.codes

/tmp/ipython-input-484408621.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all2["question_response_min_diff_buck"] = pd.qcut(all2["question_response_min_diff"], q=10).cat.codes


In [ ]:
all3 = all2.drop_duplicates("question_id")

In [ ]:
all3.iloc[:5, 55:175]

,financial-inclusion_price,financial-inclusion_sell,financial-inclusion_buy,diversification_obtain,diversification_acquire,question_what,question_when,question_how,question_which,crop_harvest,climate_time,disease_harvest,livestock_livestock,livestock_animals,livestock_sheep,livestock_cattle,livestock_donkey,livestock_pig,livestock_cow,livestock_goat,livestock_camel,livestock_hen,livestock_chicken,livestock_poultry,livestock_bee,crop_layer,diversification_expand,diversification_begin,crop_plant,crop_variety,disease_plant,crop_seed,crop_grow,crop_mulch,diversification_grow,crop_crop,crop_fruit,crop_potatoes,crop_maize,crop_banana,crop_coffee,crop_tea,crop_onions,crop_bean,crop_cabbage,climate_climate,climate_rain,diversification_add,question_who,climate_sun,diversification_use,crop_tomatoes,crop_rice,disease_disease,disease_care,crop_keep,disease_keep,financial-inclusion_market,financial-inclusion_cost,question_many,disease_rabbit,disease_treat,disease_prevent,disease_chemical,livestock_milk,crop_passion,crop_land,livestock_egg,livestock_breed,crop_fertilizer,crop_manure,diversification_increase,climate_season,disease_control,question_why,diversification_best,crop_yield,livestock_dairy,crop_soil,diversification_irrigation,diversification_scale,financial-inclusion_money,diversification_new,question_where,disease_spray,disease_turn,disease_attack,crop_type,crop_leave,disease_medicine,disease_leave,disease_tick,disease_pests,financial-inclusion_finance,financial-inclusion_investment,financial-inclusion_loan,livestock_lay,climate_heat,financial-inclusion_fee,disease_weed,diversification_clear,diversification_acreage,disease_black,disease_space,climate_drought,disease_sick,climate_temperature,climate_flood,disease_affect,financial-inclusion_sum,disease_unhealthy,financial-inclusion_insurance,financial-inclusion_bank,disease_mean,financial-inclusion_credit,diversification_register,diversification_diversify,diversification_shift,climate_water,financial-inclusion_economy
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.

In [ ]:
niche_collst = all3.columns[55:175]

In [ ]:
all3.iloc[:5, -13:-6]

,financial-inclusion,diversification,question,crop,climate,disease,livestock
0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
1,NaN,NaN,1.0,1.0,1.0,1.0,1.0
2,NaN,1.0,1.0,1.0,NaN,1.0,NaN
4,1.0,1.0,1.0,1.0,1.0,1.0,NaN
8,NaN,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
broad_collst = all3.columns[-13:-6]

# Corr heatmap

In [ ]:
niche_df = all3[niche_collst].fillna(0)
niche_df.columns = [s.split("_")[1] for s in niche_df.columns]

In [ ]:
niche_df1 = niche_df.loc[:, ~niche_df.columns.duplicated()]

In [ ]:
niche_df1.shape

(2605842, 115)

In [ ]:
corr_df = niche_df1.corr()

# Occurrence scatter

In [ ]:
topic_counts = all3[niche_collst].sum()
topic_pcts = (topic_counts / len(all3)).reset_index().set_index("index")
topic_pcts = topic_pcts.rename(columns={
  0: "pct"
})

topic_pcts = topic_pcts.reset_index()
topic_pcts["niche"] = [s.split("_")[1] for s in topic_pcts["index"]]


pairs_df = topic_pcts[["niche", "pct"]].merge(
  topic_pcts[["niche", "pct"]],
  how="cross",
  suffixes=("_1", "_2")
)

# pairs_df = pairs_df[pairs_df["index_1"] < pairs_df["index_2"]]
pairs_df["sum"] = pairs_df["pct_1"] + pairs_df["pct_2"]

# PCA

In [ ]:
from sklearn.preprocessing import StandardScaler

df_niche = all3[niche_collst]
df_niche = df_niche.fillna(0)
df_niche = df_niche.T.sort_index().T

scaler = StandardScaler()
df_niche_scaled = scaler.fit_transform(df_niche)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()

pca.fit(df_niche_scaled)

PCA()

In [ ]:
loadings = pca.components_[0:2, :].T
features_names

In [ ]:
n_components=25

pca_bigfig_dct = {
  "pc_names_lst": [f"PC{i+1}" for i in range(n_components)][::-1],
  "variance_lst": [f"{v:.2%}" for v in pca.explained_variance_ratio_[:n_components]][::-1],
  "comp_scores_lst": pca.components_[:n_components][::-1],
  "niche_collst": df_niche.columns
}

In [ ]:
pca_by_row_df = pd.DataFrame(data=pca.fit_transform(df_niche_scaled), columns=range(1, pca.fit_transform(df_niche_scaled).shape[1]+1))

In [ ]:
segment_collst = all3.columns[-5:-1]

In [ ]:
pca_by_row_segments_df = pca_by_row_df.merge(all3[segment_collst], left_index=True, right_index=True)

In [ ]:
from collections import defaultdict
pca_segments_dct = defaultdict(dict)
for idx, c in enumerate(pca_by_row_segments_df.columns[:-4]):
  neg_mk = pca_by_row_segments_df[c] <= pca_by_row_segments_df[c].quantile(0.10)
  pos_mk = pca_by_row_segments_df[c] >= pca_by_row_segments_df[c].quantile(0.90)
  neg_df = pca_by_row_segments_df[neg_mk]
  pos_df = pca_by_row_segments_df[pos_mk]

  neg_pcts = neg_df[segment_collst].apply(lambda x: x.value_counts(normalize=True)).T
  pos_pcts = pos_df[segment_collst].apply(lambda x: x.value_counts(normalize=True)).T



  pca_segments_dct[f"PC{idx+1}"]["neg"] = neg_pcts
  pca_segments_dct[f"PC{idx+1}"]["pos"] = pos_pcts


In [ ]:
pca_segments_dct["PC1"]

{'neg':                1         2         3         4         5
 Count   0.143913  0.083399  0.156847  0.313958  0.301883
 Speed   0.230962  0.240843  0.323470  0.119254  0.085470
 Reach   0.050545  0.125046  0.176977  0.149160  0.498271
 Tenure  0.062953  0.053107  0.222643  0.457642  0.203654,
 'pos':                1         2         3         4         5
 Count   0.143501  0.084665  0.159662  0.313289  0.298882
 Speed   0.232782  0.241854  0.324256  0.117479  0.083630
 Reach   0.047957  0.128235  0.178052  0.154767  0.490989
 Tenure  0.060398  0.054291  0.221025  0.462545  0.201741}

In [ ]:
n_components = 25

pc_df = pd.DataFrame(
  pca.components_[:n_components],
  index=[f"PC{idx+1}" for idx in range(n_components)],
  columns = df_niche.columns,
)

#upload data

In [ ]:
tab7_data = {
  "bigfig1": pca_bigfig_dct,
  "bigfig2": {
    "pos_neg_counts": pca_segments_dct,
    "scores": pc_df
  }
}

In [ ]:
alldata_upload = {
  # "tab6": tab6_data,
  "tab7": tab7_data
}

In [ ]:
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 204.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.0 MB/s eta 0:00:00


In [ ]:
import pickle

with open("tabs67.pkl", "wb") as wb:
  pickle.dump(alldata_upload, wb)


In [ ]:
import boto3

s3 = boto3.client("s3")

s3.upload_file(
    Filename="tabs67.pkl",
    Bucket="datakit-farmers-dashboard",
    Key="tabs67.pkl"
)